In [206]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import json


Encoding de votaciones

- 1: Si
- 2: No
- 3: Abstencion
- 4: Pareo
- -1: No disponible

In [145]:
def save_votaciones_csv(data, legislatura):
    """
    [
        {'nombre': 'Senador Apellido 1', 'vot-368-113-1': 1, 'vot-368-113-2': 0},
        {'nombre': 'Senador Apellido 2', 'vot-368-113-1': 1, 'vot-368-113-2': 1},
        {'nombre': 'Senador Apellido 3', 'vot-368-114-1': 0, 'vot-368-114-2': 1},
    ]
    Donde cada columna corresponde a una votacion con el siguiente formato
        - vot-{legiid}-{sesiid}-{numero_votacion}]
    
    """
    df = pd.DataFrame(data)
    df = df.fillna(-1)
    name = f"./{legislatura.number}.csv"
    
    df.to_csv(name, index=False)




In [204]:
def save_votaciones_json(legislaturas, file_name):
    """
    Guarda toda la informacion en un puro archivo json
    """
    all_info = []
    
    for legislatura in legislaturas:
        curr_legislatura = {}
        curr_legislatura['legiid'] = legislatura.legiid
        curr_legislatura['fechas'] = legislatura.fechas
        curr_legislatura['numero'] = legislatura.number
        sesiones = []
        
        for sesion in legislatura.sesiones:
            curr_sesion = {}
            curr_sesion['sesiid'] = sesion.sesiid
            curr_sesion['fecha'] = sesion.fecha
            curr_sesion['numero'] = sesion.number
            votaciones = []
            
            for votacion in sesion.votaciones:
                curr_votacion = {}
                curr_votacion['votaid'] = votacion.votaid
                curr_votacion['url'] = votacion.url
                curr_votacion['votos'] = votacion.votos
                curr_votacion['numero'] = votacion.numero
                
                votaciones.append(curr_votacion)
            
            
            curr_sesion['votaciones'] = votaciones
            sesiones.append(curr_sesion)
            
        

        
        curr_legislatura['sesiones'] = sesiones

        all_info.append(curr_legislatura)
        
    with open(file_name, "w", encoding='utf-8') as outfile:  
        json.dump(all_info, outfile) 
        


In [141]:
class Votacion:
    
    BASE_URL = 'https://www.senado.cl/appsenado/'
    
    def __init__(self, legiid, sesiid, votaid, url, numero):
        self.legiid = legiid
        self.sesiid = sesiid
        self.votaid = votaid
        self.url = Votacion.BASE_URL + url
        # TODO: Agregar tema + fecha
        
        # Identificador de numero de votacion en la sesion
        self.numero = numero
        
        self.votos = {}
        self.get_votos()
        
    def get_votos(self):
        answer = requests.get(self.url)
        soup = BeautifulSoup(answer.text, 'lxml')
        
        rows = soup.findAll("tr")
        
        vote_options = {0: '1', 1: '2', 3: '3', 4: '4'}
        
        
        for row in rows[1:]:
            options = row.findAll('td')
            senador = options[0].text
            valid_options = options[1:]
            
            selected = -1
            for i, option in enumerate(valid_options):
                if option.text != ' ':
                    selected = i + 1
                    break
                    
            self.votos[senador] = selected
                    
            
                    

        
    def __repr__(self):
        return f'{self.url}'

In [142]:
class Sesion:
    BASE_URL = 'https://www.senado.cl/appsenado/index.php?mo=sesionessala&ac=votacionSala&legiid='

    def __init__(self, sesiid, number, fecha, legiid):
        self.legiid = legiid
        self.sesiid = sesiid
        self.number = number
        self.fecha = fecha
        self.votaciones = []
        
        self.get_votaciones()
        
        
        
        
    def get_votaciones(self):
        
        url = Sesion.BASE_URL + self.legiid + '&sesiid=' + self.sesiid
        answer = requests.get(url)
        soup = BeautifulSoup(answer.text, 'lxml')
        
        votaciones = soup.find_all("a", string="Ver Votación")
        
        numero=1
        for votacion in votaciones:
            url = votacion['href']
            votaid = url[url.find('votaid')+7:]
            vot = Votacion(self.legiid, self.sesiid, votaid, url, numero)
            numero += 1
            
            self.votaciones.append(vot)
            
        
        
            
        
        
    def __repr__(self):
        return f'Ses: {self.number} (id: {self.sesiid}) - {self.fecha}'
        

In [174]:
class Legislatura:
    
    BASE_URL = 'https://www.senado.cl/appsenado/index.php?mo=sesionessala&ac=votacionSala&legiid='
    
    def __init__(self, legiid, number, fechas):
        # legiid for url
        self.legiid = legiid
        self.fechas = fechas
        
        
        self.number = number
        self.sesiones = []
        self.get_sesiones()
        
        
        
    def get_sesiones(self):
        url = Legislatura.BASE_URL + self.legiid
        answer = requests.get(url)
        
        
        
        soup = BeautifulSoup(answer.text, 'lxml')
        options = soup.find("select",{"name":"sesionessala"}).findAll("option")
        
        for option in tqdm(options):
            if option['value'] == '0':
                continue
            ses = Sesion(option['value'], option.text[:5], option.text[6:], self.legiid)
            self.sesiones.append(ses)
            
            ## Eliminar para obtener todas las sesiones
            # break
            
    def transform_data(self):
        """
        Despues de cargar todos los datos de las sesiones,
        Acumular por nombre
        """
        # data = {nombre: {id: voto}}
        data = {}
        
        for sesion in self.sesiones:
            for votacion in sesion.votaciones:
                legiid = self.legiid
                sesiid = sesion.sesiid
                numero = votacion.numero
                
                for nombre, voto in votacion.votos.items():
                    if nombre not in data:
                        data[nombre] = {f'votacion-{legiid}-{sesiid}-{numero}': voto}
                    else:
                        data[nombre][f'votacion-{legiid}-{sesiid}-{numero}'] = voto
                        
        data_for_csv = []
        for nombre, votos in data.items():
            formatted_data = {'nombre': nombre, **votos}
            data_for_csv.append(formatted_data)
    
        return data_for_csv
        

        
        
    def __repr__(self):
        return f'Leg: {self.number} (id: {self.legiid}) - {self.fechas}'

In [186]:
def get_legislaturas(req_answer):
    """
    Dada la pagina base, obtiene los links para todas las legislaturas
    
    
    retorna lista con legilatura
    """
    soup = BeautifulSoup(answer.text, 'lxml')
    options = soup.find("select",{"name":"legislaturas"}).findAll("option")
    
    lista = []
    
    
    for option in options:
        print(option.text[:3])
        leg = Legislatura(option['value'], option.text[:3], option.text[4:])
        lista.append(leg)
        ## Eliminar para obtener todas las legislaturas
        # break
    
    return lista
    

Al cargar una legislatura
- Busca y carga cada una de sus sesiones
    - Busca y carga todas las votaciones
    - Cargar las votaciones considera buscar sus ids y urls

In [207]:
url="https://www.senado.cl/appsenado/index.php?mo=sesionessala&ac=votacionSala&legiini=361&legiid=491"
answer = requests.get(url)

In [187]:
legislaturas = get_legislaturas(answer)

368






  0%|                                                                                          | 0/117 [00:00<?, ?it/s]



  2%|█▍                                                                                | 2/117 [00:02<02:39,  1.38s/it]



  3%|██                                                                                | 3/117 [00:05<03:34,  1.88s/it]



  3%|██▊                                                                               | 4/117 [00:08<04:10,  2.22s/it]



  4%|███▌                                                                              | 5/117 [00:12<04:46,  2.56s/it]



  5%|████▏                                                                             | 6/117 [00:14<04:39,  2.52s/it]



  6%|████▉                                                                             | 7/117 [00:17<04:47,  2.61s/it]



  7%|█████▌                                                                            | 8/117 [00:21<05:36,  3.09s/it]



  8%|██████▎

 57%|██████████████████████████████████████████████▍                                  | 67/117 [03:31<02:46,  3.32s/it]



 58%|███████████████████████████████████████████████                                  | 68/117 [03:35<02:46,  3.39s/it]



 59%|███████████████████████████████████████████████▊                                 | 69/117 [03:38<02:39,  3.32s/it]



 60%|████████████████████████████████████████████████▍                                | 70/117 [03:41<02:37,  3.34s/it]



 61%|█████████████████████████████████████████████████▏                               | 71/117 [03:45<02:42,  3.54s/it]



 62%|█████████████████████████████████████████████████▊                               | 72/117 [03:48<02:32,  3.38s/it]



 62%|██████████████████████████████████████████████████▌                              | 73/117 [03:52<02:35,  3.53s/it]



 63%|███████████████████████████████████████████████████▏                             | 74/117 [03:55<02:23,  3.33s/it]



 64%|███████████

367






  0%|                                                                                          | 0/111 [00:00<?, ?it/s]



  2%|█▍                                                                                | 2/111 [00:03<03:14,  1.78s/it]



  3%|██▏                                                                               | 3/111 [00:06<03:35,  1.99s/it]



  4%|██▉                                                                               | 4/111 [00:09<04:11,  2.35s/it]



  5%|███▋                                                                              | 5/111 [00:11<04:14,  2.40s/it]



  5%|████▍                                                                             | 6/111 [00:13<04:02,  2.31s/it]



  6%|█████▏                                                                            | 7/111 [00:18<05:02,  2.91s/it]



  7%|█████▉                                                                            | 8/111 [00:21<05:07,  2.99s/it]



  8%|██████▋

...





  0%|                                                                                           | 0/58 [00:00<?, ?it/s]



  3%|██▊                                                                                | 2/58 [00:01<00:33,  1.66it/s]



  5%|████▎                                                                              | 3/58 [00:02<00:44,  1.24it/s]



  7%|█████▋                                                                             | 4/58 [00:03<00:50,  1.07it/s]



  9%|███████▏                                                                           | 5/58 [00:04<00:54,  1.03s/it]



 10%|████████▌                                                                          | 6/58 [00:06<00:56,  1.08s/it]



 12%|██████████                                                                         | 7/58 [00:07<00:57,  1.13s/it]



 14%|███████████▍                                                                       | 8/58 [00:08<00:57,  1.15s/it]



 16%|███████

347






  0%|                                                                                           | 0/32 [00:00<?, ?it/s]



  6%|█████▏                                                                             | 2/32 [00:00<00:14,  2.05it/s]



  9%|███████▊                                                                           | 3/32 [00:01<00:18,  1.61it/s]



 12%|██████████▍                                                                        | 4/32 [00:02<00:20,  1.36it/s]



 16%|████████████▉                                                                      | 5/32 [00:03<00:21,  1.24it/s]



 19%|███████████████▌                                                                   | 6/32 [00:04<00:22,  1.18it/s]



 22%|██████████████████▏                                                                | 7/32 [00:05<00:22,  1.14it/s]



 25%|████████████████████▊                                                              | 8/32 [00:06<00:21,  1.11it/s]



 28%|███████

346






  0%|                                                                                           | 0/19 [00:00<?, ?it/s]



 11%|████████▋                                                                          | 2/19 [00:00<00:06,  2.53it/s]



 16%|█████████████                                                                      | 3/19 [00:01<00:08,  1.93it/s]



 21%|█████████████████▍                                                                 | 4/19 [00:02<00:08,  1.68it/s]



 26%|█████████████████████▊                                                             | 5/19 [00:03<00:09,  1.52it/s]



 32%|██████████████████████████▏                                                        | 6/19 [00:03<00:08,  1.44it/s]



 37%|██████████████████████████████▌                                                    | 7/19 [00:04<00:08,  1.39it/s]



 42%|██████████████████████████████████▉                                                | 8/19 [00:05<00:08,  1.36it/s]



 47%|███████

In [195]:
for legislatura in legislaturas:
    csv_data = legislatura.transform_data()
    save_votaciones_csv(data_for_csv, legislatura)


In [205]:
save_votaciones_json(legislaturas, 'all_data.json')